In [44]:
from bse_dsdm.preprocessing.feature_scaling import *
from bse_dsdm.preprocessing.outlier_detection import *
from bse_dsdm.preprocessing.optimal_outlier_detection import *
from bse_dsdm.preprocessing.value_encoder import *
from bse_dsdm.preprocessing.value_imputer import *
from bse_dsdm.preprocessing.missing_values import *

from bse_dsdm.data.data_loader import *

from bse_dsdm.model_development.model import *

from bse_dsdm.accuracy_testing.error_testing_methods import *
from bse_dsdm.accuracy_testing.cross_validation import *

LOAD DATA

In [45]:
#Load the data
data=MLDataLoader()
data.load_data("../train.csv")


PREPROCESSING

In [46]:
#Delete columns that exceed 80% of null values
data.data=remove_columns_with_na(data.data,80)


In [47]:
#Returns the categorical and numerical columns with null values
categorical_columns_na, numerical_columns_na=get_na_columns_by_type(data.data)
print(categorical_columns_na)
print(numerical_columns_na)


['club_joined', 'player_traits']
['value_eur', 'release_clause_eur', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']


In [48]:
all_cat_cols, all_numerical_cols = get_columns_by_type(data.data)
all_numerical_cols.remove('id')

In [49]:
data.data = encode_ordinal_columns(data.data,['position'])

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [50]:
X = data.data[all_numerical_cols]
y = data.data['position_encoded']
optimal_outliers = remove_optimal_outliers(X,y, accuracy_test='neg_mean_squared_error')

removing outliers with iqr method
param_val: 1.0
      overall  potential  value_eur  wage_eur  height_cm  weight_kg  \
1        65.0       71.0  1000000.0    2000.0      183.0       73.0   
2        65.0       77.0  1600000.0    2000.0      178.0       69.0   
3        72.0       72.0  2300000.0    5000.0      188.0       81.0   
5        68.0       68.0  1200000.0     500.0      183.0       70.0   
8        69.0       69.0  1300000.0    7000.0      188.0       84.0   
...       ...        ...        ...       ...        ...        ...   
6882     66.0       66.0   675000.0    1000.0      179.0       75.0   
6889     62.0       73.0   850000.0    3000.0      175.0       68.0   
6892     70.0       80.0  3400000.0    6000.0      177.0       73.0   
6893     71.0       71.0  1300000.0    7000.0      176.0       70.0   
6894     66.0       70.0  1100000.0    4000.0      188.0       74.0   

      league_level  club_jersey_number  club_contract_valid_until  weak_foot  \
1              1.0

In [51]:
print(optimal_outliers)

{'removal_type': 'iso_forest', 'data':       overall  potential  value_eur  wage_eur  height_cm  weight_kg  \
0         NaN        NaN        NaN       NaN        NaN        NaN   
1        65.0       71.0  1000000.0    2000.0      183.0       73.0   
2         NaN        NaN        NaN       NaN        NaN        NaN   
3         NaN        NaN        NaN       NaN        NaN        NaN   
4         NaN        NaN        NaN       NaN        NaN        NaN   
...       ...        ...        ...       ...        ...        ...   
6890      NaN        NaN        NaN       NaN        NaN        NaN   
6891      NaN        NaN        NaN       NaN        NaN        NaN   
6892      NaN        NaN        NaN       NaN        NaN        NaN   
6893      NaN        NaN        NaN       NaN        NaN        NaN   
6894     66.0       70.0  1100000.0    4000.0      188.0       74.0   

      league_level  club_jersey_number  club_contract_valid_until  weak_foot  \
0              NaN          

In [52]:
#Impute numerical values with the mean
data.data[all_numerical_cols]=impute_values(data.data[all_numerical_cols], impute_type='mean')

In [53]:
#impute categorical values with the mode

data.data=impute_categorical(data.data,categorical_columns_na)

          id   short_name  overall  potential   value_eur  wage_eur  \
0     216302    E. García       71         71   1400000.0     10000   
1     237867   D. Cancola       65         71   1000000.0      2000   
2     253472      E. Kahl       65         77   1600000.0      2000   
3     223994    S. Mugoša       72         72   2300000.0      5000   
4     251635  A. Țigănașu       65         65    525000.0      3000   
...      ...          ...      ...        ...         ...       ...   
6890  242007    C. Whelan       59         66    425000.0      2000   
6891  187961     Paulinho       83         83  28500000.0     61000   
6892  257234   Y. Hamache       70         80   3400000.0      6000   
6893  232511    S. Sasaki       71         71   1300000.0      7000   
6894  246750    S. Haudum       66         70   1100000.0      4000   

     birthday_date  height_cm  weight_kg                  club_name  ...  \
0       1989-12-28        176         73  Club Atlético de San Luis  ..

Features

In [54]:
#Encode ordinal columns
mapping={'work_rate': {'Low/Low': 1, 'Low/Medium': 2, 'Low/High': 3, 'Medium/Low': 4, 'Medium/Medium': 5, 'Medium/High': 6, 'High/Low': 7, 'High/Medium': 8, 'High/High': 9}}

data.data = encode_ordinal_columns(data.data, columns=['work_rate'], mapping=mapping)

In [55]:
# Target encode column  
#Each row in this column will contain the mean height cm of individuals belonging to a particular nationality
data.data['nationality_name'] = target_encode_column(data.data, 'nationality_name', 'height_cm', compute_type='mean')

In [56]:
# One-hot enconding
#Replaces categorical columns with binary columns(0s and 1s)
data.data=encode_categorical_columns(data.data,column='preferred_foot')

/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future vers

In [57]:
#The year is extracted for the birthday date and substracted for the year of the data
data.data = convert_to_age(data.data, 'birthday_date',2021)

In [58]:
#One hot encoded columns derived from the processed strings in the specified column
data.data=extract_string(data.data,'body_type')

In [59]:
#we split the data between train and test
data.split_data()


In [60]:

feature_columns=['attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure', 'defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle']
target_column=['position']
X=data.data[feature_columns]
y=data.data['position']


In [61]:
data.data=scale_features(data.data,feature_columns,'min_max')

In [62]:

model = RandomForestModel(feature_columns, target_column, data.train_data, data.test_data)
model.train()
predictions = model.predict()
print(f"predictions:{predictions}")

accuracy = model.get_accuracy(predictions, data.test_data[target_column])
print(f"Accuracy: {accuracy}")


/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a fu

predictions:['LCM' 'RCB' 'LCB' ... 'RCM' 'ST' 'GK']
Accuracy: 0.3836113125453227


/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a fu

In [63]:

model = RandomForestModel(feature_columns, target_column, data.train_data, data.test_data)

param_grid = {
    'n_estimators': [20, 100, 200],
    'max_depth': [None, 10, 15],
    'min_samples_split': [1, 3, 5],
    'min_samples_leaf': [1, 2, 4],
}


best_model = perform_grid_search(model, param_grid)


Fitting 5 folds for each of 81 candidates, totalling 405 fits


/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a fu

Best Hyperparameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 3, 'n_estimators': 200}
Best Accuracy: 0.3839735503961528
Test Accuracy with Best Hyperparameters: 0.38578680203045684


/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a fu

In [64]:
evaluate_classification_metrics(model.test_data[model.target_column], best_model.predict(model.test_data[feature_columns]))

------ Multiclass Classification Metrics Summary ------
Accuracy: 0.3858 - The proportion of correctly classified instances.
Precision: 0.3210 - Weighted average precision across all classes.
Recall: 0.3858 - Weighted average recall across all classes.
F1 Score: 0.3319 - Weighted average F1 score across all classes.


/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/andrewbennett/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a fu